<a href="https://colab.research.google.com/github/harshitadd/CrossSiloFLDemo/blob/master/PreVID_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install syft

In [ ]:
##Importing Mini Batch Data 
import os 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install pydicom 
import pydicom 
import cv2
import matplotlib.pyplot as plt 

     |████████████████████████████████| 35.5MB 83kB/s 


In [ ]:
dcm_path=os.listdir('/content/drive/My Drive/Fed_Covid/minibatch/')
dcm_data={}

alpha = 1.5 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)

ctr=0
for file in dcm_path:
  name = '/content/drive/My Drive/Fed_Covid/minibatch/' + file
  temp = pydicom.dcmread(name)
  image = temp.pixel_array
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  adjusted = cv2.resize(image,(64,64))
  dcm_data[file]=adjusted
  ctr+=1
  print(ctr)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
import csv 
labels=[]
with open('/content/drive/My Drive/Fed_Covid/stage_2_train_labels.csv','r') as file:
  reader = csv.reader(file)
  for row in reader:
    labels.append(row)

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
scaler = MinMaxScaler()
pid=[]
dicom=[]
label=[]
cid = 0
for PID in labels:
  for key in dcm_data:
    if(key[:-4]==PID[0]):
      l=[]
      for val in dcm_data[key]:
        l.append(scaler.fit_transform(val))
      l = np.reshape(l,(3,64,64))
      dicom.append(l)
      label.append(int(PID[5]))
      

In [ ]:
import numpy as np
print(np.shape(dicom[0]))

(3, 64, 64)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dicom,label,test_size=0.3)

In [ ]:
import pandas as pd 
train_df = pd.DataFrame()
train_df['features']=x_train
train_df['labels']=y_train

In [ ]:
train_df

,features,labels
0,"[[[0.0, 0.0, 0.0, 0.152, 0.152, 0.152, 0.04, 0...",0
1,"[[[1.0, 1.0, 1.0, 0.39436619718309857, 0.39436...",1
2,"[[[0.005208333333333333, 0.005208333333333333,...",0
3,"[[[0.010526315789473684, 0.010526315789473684,...",0
4,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0
...,...,...
295,"[[[0.025316455696202535, 0.025316455696202535,...",0
296,"[[[0.013333333333333332, 0.013333333333333332,...",1
297,"[[[0.011299435028248588, 0.011299435028248588,...",0
298,"[[[0.015151515151515152, 0.015151515151515152,...",0


In [ ]:
test_df = pd.DataFrame()
test_df['features']=x_test
test_df['labels']=y_test

In [ ]:
class data_maker(Dataset):
  def __init__(self, data):
        self.data = data
        
  def __len__(self):
      return len(self.data)
  
  def __getitem__(self, index):
      target = np.array(self.data['labels'][index])
      data_val = np.array(self.data['features'][index])

      return data_val,target
      
        

In [ ]:
train_data = data_maker(train_df)
test_data = data_maker(test_df)

In [ ]:
def get_train_data(batch_size):
      train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)
      return train_loader

In [ ]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4)


In [ ]:
train_loader= get_train_data(batch_size=8)
n_batches = len(train_loader)
for i, data in enumerate(train_loader, 0):
       inputs, labels = data
       print(np.shape(inputs))
       break   

torch.Size([8, 3, 64, 64])


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Input - (32,1024,1024)
  
        self.conv1 = nn.Conv2d(3,32, kernel_size=8, stride = 2)  #I/P - 1*32
        self.pool1 = nn.MaxPool2d(kernel_size=4,stride=2,padding=0)  
        self.conv2 = nn.Conv2d(32, 64, 8)
        self.pool2 = nn.MaxPool2d(4,2,padding=0)
        self.fc1 = nn.Linear(256, 64*64)
        self.fc2 = nn.Linear(64*64, 64)
        self.fc3 = nn.Linear(64, 8)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        # Input of 1*1024*1024 going to 32*1024*1024  - 1*64*64 going to 32*64*64
        x = self.pool1(x)
        # Input of 32*1024*1024 going to 32*256*256 - 32*64*64 going to 32*16*16
        x = F.relu(self.conv2(x))
        # Input of 32*256*256 to 64*256*256 - 32*16*16 to 64*16*16
        x = self.pool2(x)
        # Input of 64*256*256 to 64*64*64 - 64*16*16 going to 64*4*4

        x = x.view(-1,64*4)
     
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [ ]:
import time
from torch.autograd import Variable
def trainNet(net, batch_size, n_epochs, learning_rate):
    
    
    #Get training data
    train_loader = get_train_data(batch_size=8)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
          
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            

            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            

            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [ ]:
CNN = Net()
trainNet(CNN.double(), batch_size=8, n_epochs=2
         , learning_rate=0.001)

Epoch 1, 10% 	 train_loss: 1.95 took: 0.51s
Epoch 1, 21% 	 train_loss: 1.39 took: 0.19s
Epoch 1, 31% 	 train_loss: 0.64 took: 0.19s
Epoch 1, 42% 	 train_loss: 1.13 took: 0.19s
Epoch 1, 52% 	 train_loss: 0.77 took: 0.19s
Epoch 1, 63% 	 train_loss: 0.79 took: 0.20s
Epoch 1, 73% 	 train_loss: 0.62 took: 0.19s
Epoch 1, 84% 	 train_loss: 0.57 took: 0.19s
Epoch 1, 94% 	 train_loss: 0.67 took: 0.19s
Epoch 2, 10% 	 train_loss: 0.64 took: 0.20s
Epoch 2, 21% 	 train_loss: 0.82 took: 0.20s
Epoch 2, 31% 	 train_loss: 0.49 took: 0.19s
Epoch 2, 42% 	 train_loss: 0.97 took: 0.19s
Epoch 2, 52% 	 train_loss: 0.74 took: 0.19s
Epoch 2, 63% 	 train_loss: 0.74 took: 0.21s
Epoch 2, 73% 	 train_loss: 0.55 took: 0.20s
Epoch 2, 84% 	 train_loss: 0.54 took: 0.20s
Epoch 2, 94% 	 train_loss: 0.67 took: 0.19s
Training finished, took 4.02s


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = CNN(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('Accuracy of the network on test images: %d %%' % (100 * correct / total))

Accuracy of the network on test images: 81 %


https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html - Test Loader

PYTORCH IMPLEMENTATION


In [ ]:
import syft as sy 
hook = sy.TorchHook(torch)

In [ ]:
hospital = sy.VirtualWorker(hook, id="hospital")  
clinic = sy.VirtualWorker(hook, id="clinic")  

In [ ]:
print(x_train[0])

[[[0.10989011 0.10989011 0.10989011 ... 0.34065934 0.34065934 0.37362637]
  [0.37362637 0.37362637 0.40659341 ... 0.42307692 0.37912088 0.37912088]
  [0.37912088 0.31868132 0.31868132 ... 0.02197802 0.02197802 0.02197802]
  ...
  [0.13194444 0.13194444 0.19444444 ... 0.09722222 0.02777778 0.02777778]
  [0.02777778 0.         0.         ... 0.91666667 0.91666667 0.91666667]
  [0.96376812 0.96376812 0.96376812 ... 0.15217391 0.15217391 0.1884058 ]]

 [[0.1884058  0.1884058  0.37681159 ... 0.02898551 0.08695652 0.08695652]
  [0.08695652 0.         0.         ... 0.89130435 0.89130435 0.89130435]
  [0.95419847 0.95419847 0.95419847 ... 0.23664122 0.23664122 0.18320611]
  ...
  [0.76422764 0.71544715 0.71544715 ... 0.71544715 0.71544715 0.71544715]
  [0.776      0.776      0.776      ... 0.304      0.304      0.264     ]
  [0.264      0.264      0.256      ... 0.76       0.768      0.768     ]]

 [[0.768      0.696      0.696      ... 0.656      0.656      0.656     ]
  [0.74590164 0.745901

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)

In [ ]:
#define the send_data function 
data_hospital = x_train[0:80].send(hospital)
data_target_hospital = y_train[0:80].send(hospital)
data_clinic = x_train[80:].send(clinic)
data_target_clinic = y_train[80:].send(clinic)

In [ ]:
datasets=[(data_hospital,data_target_hospital),(data_clinic,data_target_clinic)]

In [ ]:
opt = optim.SGD(params=model.parameters(),lr=0.1)

In [ ]:
# def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    
    
#     #Optimizer
#     optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
#     return(loss, optimizer)

##Stateful issue fix - https://github.com/OpenMined/PySyft/pull/3179/files

In [ ]:
import copy


class Optims:
    """to create a list of optimizer objects"""

    def __init__(self, workers, optim):
        """
        args:
            workers: list of worker ids
            optim: class of pytorch optimizer
        """
        self.optim = optim
        self.workers = workers
        self.optimizers = {}
        for worker in workers:
            self.optimizers[str(worker)] = copy.copy(self.optim)
            self.optimizers[str(worker)].load_state_dict((self.optim).state_dict())

    def get_optim(self, worker):
        """returns optimizer for worker
        args: 
            worker: worker id
        """
        return self.optimizers[str(worker)]

    def count(self):
        """returns the number of optimizer objects"""
        return len(self.workers)


In [ ]:
import time
from torch.autograd import Variable
from syft.federated.floptimizer import Optims
workers = ['hospital', 'clinic']

def federated_train_net(model):

  # loss, optimizer = createLossAndOptimizer(model)
  training_start_time = time.time()
    

  for epoch in range(20):
    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0
    loss = torch.nn.CrossEntropyLoss()
    for _data, _target in datasets:

      ##send model 
      model = model.send(_data.location)
      
      ##Getting stateful optimizer
      optims = Optims(workers, optim=optim.Adam(params=model.parameters(),lr=0.001)) 
      opt = optims.get_optim(_data.location.id)
      opt.zero_grad()
      

      #Forward pass, backward pass, optimize
      outputs = model(_data)
      loss_size = loss(outputs, _target)
      loss_size.backward()
      opt.step()

      #get the model back
      model = model.get()
      
      #Print statistics
      print('Training loss for epoch ' + str(epoch) + str(loss_size.get()) + str(_data.location))
      # running_loss =running_loss + loss_size.data.clone().get()
      # total_train_loss =total_train_loss+ loss_size.data
      
      # #Print every 10th batch of an epoch
      # if (i + 1) % (print_every + 1) == 0:
      #     print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
      #             epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
      #     #Reset running loss and time
      #     running_loss = 0.0
      #     start_time = time.time()


      # print('Training loss for epoch ' + str(epoch) + str(loss.get()) + str(_data.location))

In [ ]:
CNN = Net()
federated_train_net(CNN.double())

Training loss for epoch 0tensor(2.1360, dtype=torch.float64, requires_grad=True)<VirtualWorker id:hospital #objects:100>
Training loss for epoch 0tensor(1.6760, dtype=torch.float64, requires_grad=True)<VirtualWorker id:clinic #objects:115>
Training loss for epoch 1tensor(0.6273, dtype=torch.float64, requires_grad=True)<VirtualWorker id:hospital #objects:100>
Training loss for epoch 1tensor(0.5262, dtype=torch.float64, requires_grad=True)<VirtualWorker id:clinic #objects:115>
Training loss for epoch 2tensor(1.0784, dtype=torch.float64, requires_grad=True)<VirtualWorker id:hospital #objects:100>
Training loss for epoch 2tensor(0.7287, dtype=torch.float64, requires_grad=True)<VirtualWorker id:clinic #objects:115>
Training loss for epoch 3tensor(0.8410, dtype=torch.float64, requires_grad=True)<VirtualWorker id:hospital #objects:100>
Training loss for epoch 3tensor(0.6805, dtype=torch.float64, requires_grad=True)<VirtualWorker id:clinic #objects:115>
Training loss for epoch 4tensor(0.8140, 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = CNN(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('Accuracy of the network on test images: %d %%' % (100 * correct / total))

Accuracy of the network on test images: 78 %
